In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [ ]:
'''Some of the next “data cleaning” steps included

Replacing -9999, -666666666, and -99999999’s with NaNs
Inverting the sign of the variables that are negatively correlated with vulnerability
(median housing value, median gross rent, Asian, median household income, population with bachelors degree, and per capita income
Taking the average of repeated data points
Normalizing the data to the same scale. I used a “vectorwise z-score of the data in A with center 0 and standard deviation 1”
Merging the now – cleaned ACS data with a GIS dataset of census tracts using “joins and relates”. This way the ACS data can be evaluated in GIS.
 
 '''

In [2]:

# Load the data
df = pd.read_csv('acs5_2023_tx_tract_data.csv')

# Step 1: Replace missing value placeholders with NaN
missing_values = [-9999, -666666666, -99999999]
df.replace(missing_values, np.nan, inplace=True)


In [3]:
df

,DP05_0024PE,DP05_0018E,DP03_0066PE,DP02_0061PE,DP03_0009PE,DP03_0033PE,DP03_0028PE,DP05_0070PE,DP04_0076E,DP05_0003PE,...,DP03_0088E,DP02_0115PE,DP04_0058PE,DP05_0044PE,DP05_0038PE,DP05_0039PE,DP03_0062E,state,county,tract
0,16.2,42.6,39.1,4.1,6.3,6.6,11.0,7.6,1927.0,49.4,...,32490.0,1.5,2.5,0.0,6.0,0.0,67344.0,48.0,1.0,950100.0
1,3.4,38.8,23.1,23.3,0.0,0.0,31.0,36.4,26.0,0.6,...,3462.0,8.1,0.0,0.0,34.6,0.6,98750.0,48.0,1.0,950401.0
2,1.9,42.8,NaN,20.5,NaN,NaN,NaN,36.2,0.0,0.0,...,1472.0,5.7,NaN,0.0,34.8,0.1,NaN,48.0,1.0,950402.0
3,18.0,42.1,48.6,7.3,5.9,1.3,18.5,18.7,1425.0,48.9,...,25685.0,12.7,8.8,0.0,17.8,0.6,59358.0,48.0,1.0,950500.0
4,14.1,35.1,31.2,8.5,3.0,5.8,19.1,15.2,1948.0,50.9,...,28413.0,1.6,12.0,0.0,14.7,0.0,44250.0,48.0,1.0,950600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,20.6,39.2,57.2,13.6,0.0,23.0,16.0,1.4,824.0,53.9,...,24964.0,35.8,19.2,0.0,0.8,0.0,36250.0,48.0,505.0,950402.0
6892,13.1,26.4,28.6,17.9,0.0,5.7,48.2,0.0,332.0,42.1,...,14357.0,5.6,9.9,0.0,0.0,0.0,26227.0,48.0,507.0,950100.0
6893,11.9,27.7,20.7,21.4,7.9,9.8,11.8,6.2,411.0,45.8,...,20589.0,16.9,0.0,0.0,6.2,0.9,42270.0,48.0,507.0,950200.0
6894,24.5,34.3,25.0,16.5,21.4,6.5,10.0,0.2,869.0,45.8,...,27967.0,20.7,7.2,0.0,0.0,0.0,52120.0,48.0,507.0,950301.0


In [4]:

# Step 2: Invert signs of negatively correlated variables
# Relevant ACS variable codes from your image (excluding 'skip')
relevant_codes = [
    'DP05_0024PE', 'DP05_0018E', 'DP03_0066PE', 'DP02_0061PE', 'DP03_0009PE',
    'DP03_0033PE', 'DP03_0028PE', 'DP05_0070PE', 'DP04_0076E', 'DP05_0003PE',
    'DP03_0010PE', 'DP03_0099PE', 'DP04_0047PE', 'DP04_0134E', 'DP04_0014PE',
    'DP04_0003PE', 'DP03_0119PE', 'DP03_0061PE', 'DP03_0088E', 'DP02_0115PE',
    'DP04_0058PE', 'DP05_0044PE', 'DP05_0038PE', 'DP05_0039PE', 'DP03_0062E',
    'state', 'county', 'tract'
]


# Keep only the relevant columns
df = df[[col for col in relevant_codes if col in df.columns]]





In [7]:
# Variables negatively correlated with vulnerability — to be inverted
neg_corr_vars = [
    'DP03_0062E',  # Median housing value
    'DP04_0134E',  # Median gross rent
    'DP05_0044PE', # Percent Asian
    'DP03_0088E',  # Per capita income
    'DP03_0061PE'  # Percent earning >200k
    # Add bachelor’s degree variable - no code -ask John
]

for col in neg_corr_vars:
    if col in df.columns:
        df[col] = -df[col]


In [9]:
# Group by geographic identifiers and average duplicates
group_cols = ['state', 'county', 'tract']
df = df.groupby(group_cols).mean(numeric_only=True).reset_index()


In [10]:
df

,state,county,tract,DP05_0024PE,DP05_0018E,DP03_0066PE,DP02_0061PE,DP03_0009PE,DP03_0033PE,DP03_0028PE,...,DP04_0003PE,DP03_0119PE,DP03_0061PE,DP03_0088E,DP02_0115PE,DP04_0058PE,DP05_0044PE,DP05_0038PE,DP05_0039PE,DP03_0062E
0,48.0,1.0,950100.0,16.2,42.6,39.1,4.1,6.3,6.6,11.0,...,17.8,16.3,-4.7,-32490.0,1.5,2.5,0.0,6.0,0.0,-67344.0
1,48.0,1.0,950401.0,3.4,38.8,23.1,23.3,0.0,0.0,31.0,...,50.9,0.0,0.0,-3462.0,8.1,0.0,0.0,34.6,0.6,-98750.0
2,48.0,1.0,950402.0,1.9,42.8,NaN,20.5,NaN,NaN,NaN,...,NaN,NaN,NaN,-1472.0,5.7,NaN,0.0,34.8,0.1,NaN
3,48.0,1.0,950500.0,18.0,42.1,48.6,7.3,5.9,1.3,18.5,...,12.5,14.0,-3.9,-25685.0,12.7,8.8,0.0,17.8,0.6,-59358.0
4,48.0,1.0,950600.0,14.1,35.1,31.2,8.5,3.0,5.8,19.1,...,12.1,22.7,-3.2,-28413.0,1.6,12.0,0.0,14.7,0.0,-44250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,48.0,505.0,950402.0,20.6,39.2,57.2,13.6,0.0,23.0,16.0,...,35.5,8.6,-7.8,-24964.0,35.8,19.2,0.0,0.8,0.0,-36250.0
6892,48.0,507.0,950100.0,13.1,26.4,28.6,17.9,0.0,5.7,48.2,...,20.2,38.3,0.0,-14357.0,5.6,9.9,0.0,0.0,0.0,-26227.0
6893,48.0,507.0,950200.0,11.9,27.7,20.7,21.4,7.9,9.8,11.8,...,18.0,24.1,0.0,-20589.0,16.9,0.0,0.0,6.2,0.9,-42270.0
6894,48.0,507.0,950301.0,24.5,34.3,25.0,16.5,21.4,6.5,10.0,...,12.4,23.8,0.0,-27967.0,20.7,7.2,0.0,0.0,0.0,-52120.0


In [11]:
# Normalize (z-score standardization)
id_cols = ['state', 'county', 'tract']
features = df.drop(columns=id_cols)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
# Reassemble final DataFrame
df_cleaned = pd.concat([df[id_cols], pd.DataFrame(features_scaled, columns=features.columns)], axis=1)


In [12]:
# Save to CSV
df_cleaned.to_csv("acs5_2023_tx_tract_data_cleaned.csv", index=False)

print("Cleaned CSV file saved as 'acs5_2023_tx_tract_data_cleaned.csv'")

Cleaned CSV file saved as 'acs5_2023_tx_tract_data_cleaned.csv'
